# Learn Posture

use machine learning to recognize robot's posture (following the example in [scikit-learn-intro.ipynb](./scikit-learn-intro.ipynb) )

## 1. Data collection

We have colleceted data before, you need to add new data if you want to add new posture.

* the dateset are in *robot_pose_data* folder
* each file contains the data belongs to this posture, e.g. the data in *Back* file are collected when robot was in "Back" posture
* the data file can be load by ```pickle```, e.g. ```pickle.load(open('Back'))```, the data is a list of feature data
* the features (e.g. each row of the data) are ['AngleX', 'AngleY', 'LHipYawPitch', 'LHipRoll', 'LHipPitch', 'LKneePitch', 'RHipYawPitch', 'RHipRoll', 'RHipPitch', 'RKneePitch'], where 'AngleX' and 'AngleY' are body angle (e.g. ```Perception.imu```) and others are joint angles.

## 2. Data preprocessing

In [1]:
%pylab inline
import pickle
from os import listdir, path
import numpy as np
from sklearn import svm, metrics

ROBOT_POSE_DATA_DIR = 'robot_pose_data'

Populating the interactive namespace from numpy and matplotlib


In [2]:
classes = listdir(ROBOT_POSE_DATA_DIR)
print classes

['Stand', 'Left', 'Crouch', 'Sit', 'Belly', 'Knee', 'StandInit', 'Back', 'Right', 'Frog', 'HeadBack']


In [3]:
def load_pose_data(i):
    '''load pose data from file'''
    data = []
    target = []
    # YOUR CODE HERE
    
    filename = path.join(ROBOT_POSE_DATA_DIR, classes[i])
    data = pickle.load(open(filename))
    target = [i] * len(data)
    print len(data), len(target)
    return data, target

In [4]:
# load all the data
all_data = []
all_target = []

for i in range(0,len(classes)):
    data,target = load_pose_data(i)
    all_data.extend(data)
    all_target.extend(target)
    
    
print 'total number of data', len(all_data)

11 11
20 20
30 30
26 26
19 19
10 10
52 52
23 23
11 11
10 10
10 10
total number of data 222


## 3. Learn on training data

In scikit-learn, an estimator for classification is a Python object that implements the methods fit(X, y) and predict(T). An example of an estimator is the class sklearn.svm.SVC that implements support vector classification.

In [5]:
# shuffule data
permutation = np.random.permutation(len(all_data))
n_training_data = int(len(all_data) * 0.7)
training_data = permutation[:n_training_data]

In [6]:
print training_data

[ 44  83  41 162  10 219 209 183  70 210  12  87  80 125 192  60  17 124
  62 146 182 111  48  74 157  52 171  34 110 165  92 119 193 204 147 130
   0  23   1 123  40  50 134  95  78 127  54  18  14  27  90  46 199 211
  94  59  39  26  98 153   8 128 207 137 126  33  56 152  47  31  73 181
 187 172  71 180 191 173  21  49  37  77  36   5  67 166 160 164 189 113
  91 148 205 145 102 177 215 175  55 188  84 195  66  65 115 109 133 143
 169 212   3  89  42 198 170 218 140 100  81 138  53 176  99  72 206 121
 105 167 149  35   2 135  30  19 118 142  88 155  57 168  96  32  97 106
  38 122 174  45  51 161 201 208  86   7   9]


In [7]:
clf = svm.SVC(gamma=0.001, C=100.)

### learning

In [8]:
fit_data = []
fit_target = []
for i in range(0,len(training_data)):
    fit_data.append(all_data[training_data[i]])
    fit_target.append(all_target[training_data[i]])
    
clf.fit(fit_data, fit_target) 

SVC(C=100.0, cache_size=200, class_weight=None, coef0=0.0, degree=3,
  gamma=0.001, kernel='rbf', max_iter=-1, probability=False,
  random_state=None, shrinking=True, tol=0.001, verbose=False)

### predicting

In [9]:
clf.predict(all_data[1]), all_target[1]

(array([0]), 0)

In [10]:
def evaluate(expected, predicted):
    print("Classification report:\n%s\n" % metrics.classification_report(expected, predicted))

    print("Confusion matrix:\n%s" % metrics.confusion_matrix(expected, predicted))

## 4. Evaluate on the test data

In [11]:
expected = []
predicted = []
prediction_data = []
for i in range(0,len(all_data)):
    if( not (i in training_data)):
        expected.append(all_target[i])
        prediction_data.append(all_data[i])
predicted = clf.predict(prediction_data)

evaluate(expected, predicted)

Classification report:
             precision    recall  f1-score   support

          0       1.00      1.00      1.00         2
          1       1.00      1.00      1.00        10
          2       1.00      1.00      1.00         2
          3       1.00      1.00      1.00        10
          4       1.00      1.00      1.00         4
          5       1.00      1.00      1.00         4
          6       1.00      1.00      1.00        17
          7       1.00      1.00      1.00         6
          8       1.00      1.00      1.00         4
          9       1.00      1.00      1.00         2
         10       1.00      1.00      1.00         6

avg / total       1.00      1.00      1.00        67


Confusion matrix:
[[ 2  0  0  0  0  0  0  0  0  0  0]
 [ 0 10  0  0  0  0  0  0  0  0  0]
 [ 0  0  2  0  0  0  0  0  0  0  0]
 [ 0  0  0 10  0  0  0  0  0  0  0]
 [ 0  0  0  0  4  0  0  0  0  0  0]
 [ 0  0  0  0  0  4  0  0  0  0  0]
 [ 0  0  0  0  0  0 17  0  0  0  0]
 [ 0  0  0  0 

/usr/local/lib/python2.7/dist-packages/numpy/core/fromnumeric.py:2641: VisibleDeprecationWarning: `rank` is deprecated; use the `ndim` attribute or function instead. To find the rank of a matrix see `numpy.linalg.matrix_rank`.
  VisibleDeprecationWarning)


## 5. Deploy to the real system

We can simple use `pickle` module to serialize the trained classifier.

In [12]:
import pickle
ROBOT_POSE_CLF = 'robot_pose.pkl'
pickle.dump(clf, open(ROBOT_POSE_CLF, 'w'))

Then, in the application we can load the trained classifier again.

In [13]:
clf2 = pickle.load(open(ROBOT_POSE_CLF))
for i in range(0,len(all_data),5):
    print classes[clf2.predict(all_data[i])], classes[all_target[i]]

Stand Stand
Stand Stand
Stand Stand
Left Left
Left Left
Left Left
Left Left
Crouch Crouch
Crouch Crouch
Crouch Crouch
Crouch Crouch
Crouch Crouch
Crouch Crouch
Sit Sit
Sit Sit
Sit Sit
Sit Sit
Sit Sit
Belly Belly
Belly Belly
Belly Belly
Belly Belly
Knee Knee
Knee Knee
StandInit StandInit
StandInit StandInit
StandInit StandInit
StandInit StandInit
StandInit StandInit
StandInit StandInit
StandInit StandInit
StandInit StandInit
StandInit StandInit
StandInit StandInit
Back Back
Back Back
Back Back
Back Back
Back Back
Right Right
Right Right
Frog Frog
Frog Frog
HeadBack HeadBack
HeadBack HeadBack


/usr/local/lib/python2.7/dist-packages/ipykernel/__main__.py:3: DeprecationWarning: converting an array with ndim > 0 to an index will result in an error in the future
  app.launch_new_instance()
